In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization, LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import keras

In [40]:
train = pd.read_csv("trainLabels.csv")
train["image_names"] = train.apply(lambda row: str(row["id"])+".png",axis=1)
print(train.head())
print(train.info())

   id       label image_names
0   1        frog       1.png
1   2       truck       2.png
2   3       truck       3.png
3   4        deer       4.png
4   5  automobile       5.png
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           40000 non-null  int64 
 1   label        40000 non-null  object
 2   image_names  40000 non-null  object
dtypes: int64(1), object(2)
memory usage: 937.6+ KB
None


In [41]:
y_train = train["label"]
X_train = train.drop(labels = ["image_names"], axis = 1)
y_train.head()

0          frog
1         truck
2         truck
3          deer
4    automobile
Name: label, dtype: object

In [42]:
datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.2, rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,vertical_flip=False)
train_generator=datagen.flow_from_dataframe(
dataframe=train,
directory="train/train/",
x_col="image_names",
y_col="label",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

Found 32000 validated image filenames belonging to 10 classes.


In [43]:
valid_generator=datagen.flow_from_dataframe(
dataframe=train,
directory="train/train/",
x_col="image_names",
y_col="label",
subset="validation",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(32,32))

Found 8000 validated image filenames belonging to 10 classes.


In [44]:
test=pd.read_csv("sampleSubmission.csv",dtype=str)
test["image_names"] = test.apply(lambda row: str(row["id"])+".png",axis=1)
print(test.head())

      id     label image_names
0  40001     truck   40001.png
1  40002       dog   40002.png
2  40003       dog   40003.png
3  40004     horse   40004.png
4  40005  airplane   40005.png


In [45]:
test_datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.2, rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,vertical_flip=False)
test_generator=test_datagen.flow_from_dataframe(
dataframe=test,
directory="test/test/",
x_col="image_names",
y_col=None,
batch_size=32,
seed=42,
shuffle=False,
class_mode=None,
target_size=(32,32))

Found 10000 validated image filenames.


In [99]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(256, (3, 3), padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(256, (3, 3), padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999),loss="categorical_crossentropy",metrics=["accuracy"])

In [100]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50
          
          
)

Epoch 1/50
1000/1000 [==============================] - 293s 292ms/step - loss: 2.2193 - accuracy: 0.1441 - val_loss: 1.7347 - val_accuracy: 0.3280
Epoch 2/50
1000/1000 [==============================] - 286s 286ms/step - loss: 1.7451 - accuracy: 0.3269 - val_loss: 1.5043 - val_accuracy: 0.4252
Epoch 3/50
1000/1000 [==============================] - 279s 279ms/step - loss: 1.5457 - accuracy: 0.4211 - val_loss: 1.4179 - val_accuracy: 0.4557
Epoch 4/50
1000/1000 [==============================] - 274s 274ms/step - loss: 1.4320 - accuracy: 0.4737 - val_loss: 1.2680 - val_accuracy: 0.5386
Epoch 5/50
1000/1000 [==============================] - 284s 284ms/step - loss: 1.3316 - accuracy: 0.5245 - val_loss: 1.1915 - val_accuracy: 0.5591
Epoch 6/50
1000/1000 [==============================] - 281s 281ms/step - loss: 1.2506 - accuracy: 0.5546 - val_loss: 1.1300 - val_accuracy: 0.5936
Epoch 7/50
1000/1000 [==============================] - 283s 283ms/step - loss: 1.1839 - accuracy: 0.5785 - val_

KeyboardInterrupt: 

In [68]:
model.evaluate(valid_generator)

250/250 [==============================] - 40s 162ms/step - loss: 1.5230 - accuracy: 0.7744


[1.5230351686477661, 0.7743750214576721]

In [21]:
test_generator.reset()
pred=model.predict(test_generator,verbose=1)

313/313 [==============================] - 76s 243ms/step


In [107]:
print(len(pred))
predicted_class_indices=np.argmax(pred,axis=1)
print(len(predicted_class_indices))

10000
10000


In [108]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [109]:
filenames=test_generator.filenames
filenames = [i.split(".")[0] for i in filenames]
print(len(predictions))
print(len(filenames))
results=pd.DataFrame({"id":filenames,
                       "label":predictions})
results
results.to_csv("results.csv",index=False)

10000
10000


In [66]:
# Model 2
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(32,32,3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(512, activation='linear'))
model.add(LeakyReLU(alpha=0.1))      
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = optimizers.Adam(lr=0.0006, beta_1=0.9, beta_2=0.999, decay=0.0)

model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#model.compile(optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999),loss="categorical_crossentropy",metrics=["accuracy"])

In [59]:
# Model 3
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(32,32,3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.25))
model.add(Conv2D(256, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='linear'))
model.add(LeakyReLU(alpha=0.1))      
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
adam = optimizers.Adam(lr=0.0006, beta_1=0.9, beta_2=0.999, decay=0.0)

model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
# model 4
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu',  padding='same', input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu',  padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu',  padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.3))
model.add(Conv2D(128, (3, 3), activation='relu',  padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Dropout(0.4))
model.add(Conv2D(128,(3,3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv2D(128,(3,3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv2D(512,(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Conv2D(512,(3,3), activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Flatten())   
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.6))            
model.add(Dense(10, activation='softmax'))
opt = optimizers.Adam(lr = 0.001, decay = 1e-5)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [33]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=50
)

Epoch 1/50
1000/1000 [==============================] - 646s 645ms/step - loss: 2.6588 - accuracy: 0.1985 - val_loss: 1.9069 - val_accuracy: 0.3479
Epoch 2/50
1000/1000 [==============================] - 668s 668ms/step - loss: 1.7035 - accuracy: 0.3743 - val_loss: 1.4375 - val_accuracy: 0.4699
Epoch 3/50
1000/1000 [==============================] - 627s 627ms/step - loss: 1.4779 - accuracy: 0.4646 - val_loss: 1.2574 - val_accuracy: 0.5458
Epoch 4/50
1000/1000 [==============================] - 633s 633ms/step - loss: 1.3469 - accuracy: 0.5212 - val_loss: 1.1388 - val_accuracy: 0.5951
Epoch 5/50
1000/1000 [==============================] - 352s 352ms/step - loss: 1.2155 - accuracy: 0.5748 - val_loss: 1.0178 - val_accuracy: 0.6438
Epoch 6/50
1000/1000 [==============================] - 456s 456ms/step - loss: 1.1336 - accuracy: 0.6065 - val_loss: 1.0372 - val_accuracy: 0.6405
Epoch 7/50
1000/1000 [==============================] - 720s 720ms/step - loss: 1.0755 - accuracy: 0.6262 - val_

In [34]:
model.evaluate(valid_generator)

250/250 [==============================] - 26s 103ms/step - loss: 0.4263 - accuracy: 0.8541


[0.4262973666191101, 0.8541250228881836]

In [35]:
test_generator.reset()
pred=Leaves_model.predict(test_generator,verbose=1)

313/313 [==============================] - 131s 418ms/step


In [36]:
print(len(pred))
predicted_class_indices=np.argmax(pred,axis=1)
print(len(predicted_class_indices))

10000
10000


In [37]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [38]:
filenames=test_generator.filenames
filenames = [i.split(".")[0] for i in filenames]
print(len(predictions))
print(len(filenames))
results=pd.DataFrame({"id":filenames,
                       "label":predictions})
results.to_csv("results.csv",index=False)

10000
10000


**Comments and Observations:**

Keras model provided better accuracy on increasing the convolutional layer and by MaxPooling. Initially model gave an accuracy of 74% which got improved by 85%. 